# Leveraging Assistants API for SQL Databases

## Setup

In [1]:
from openai import AzureOpenAI
import json
import os

## Import the helper function


In [2]:
# to use the sql functions defined in previous step "Function Calling"
import Helper
from Helper import get_positive_cases_for_state_on_date
from Helper import get_hospitalized_increase_for_state_on_date

## Launch the Assistant API

In [3]:
client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_KEY"),
    api_version="2024-02-15-preview",
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
    )

# I) Create assistant
assistant = client.beta.assistants.create(
  instructions="""You are an assistant answering questions 
                  about a Covid dataset.""",
  model="gpt-4-1106", 
  tools=Helper.tools_sql)

# II) Create thread
thread = client.beta.threads.create()
print(thread)

Thread(id='thread_wsxhkqX5xbIs0l4Aigl4OpsF', created_at=1718639578, metadata={}, object='thread')


In [4]:
# III) Add message
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="""how many hospitalized people we had in Alaska
               the 2021-03-05?"""
)
print(message)

ThreadMessage(id='msg_j5e3kwMb3vnn5drHD2yjDVgF', assistant_id=None, content=[MessageContentText(text=Text(annotations=[], value='how many hospitalized people we had in Alaska\n               the 2021-03-05?'), type='text')], created_at=1718639643, file_ids=[], metadata={}, object='thread.message', role='user', run_id=None, thread_id='thread_wsxhkqX5xbIs0l4Aigl4OpsF')


In [5]:
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

print(messages.model_dump_json(indent=2))

{
  "data": [
    {
      "id": "msg_j5e3kwMb3vnn5drHD2yjDVgF",
      "assistant_id": null,
      "content": [
        {
          "text": {
            "annotations": [],
            "value": "how many hospitalized people we had in Alaska\n               the 2021-03-05?"
          },
          "type": "text"
        }
      ],
      "created_at": 1718639643,
      "file_ids": [],
      "metadata": {},
      "object": "thread.message",
      "role": "user",
      "run_id": null,
      "thread_id": "thread_wsxhkqX5xbIs0l4Aigl4OpsF"
    }
  ],
  "object": "list",
  "first_id": "msg_j5e3kwMb3vnn5drHD2yjDVgF",
  "last_id": "msg_j5e3kwMb3vnn5drHD2yjDVgF",
  "has_more": false
}


In [6]:
# IV) Run assistant on thread

run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
)

## Leverage the function calling with Assistants API

In [7]:
import time
from IPython.display import clear_output

start_time = time.time()

status = run.status

while status not in ["completed", "cancelled", "expired", "failed"]:
    time.sleep(5)
    run = client.beta.threads.runs.retrieve(
        thread_id=thread.id,run_id=run.id
    )
    print("Elapsed time: {} minutes {} seconds".format(
        int((time.time() - start_time) // 60),
        int((time.time() - start_time) % 60))
         )
    status = run.status
    print(f'Status: {status}')
    if (status=="requires_action"):
        available_functions = {
            "get_positive_cases_for_state_on_date": get_positive_cases_for_state_on_date,
            "get_hospitalized_increase_for_state_on_date":get_hospitalized_increase_for_state_on_date
        }

        tool_outputs = []
        for tool_call in run.required_action.submit_tool_outputs.tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            function_response = function_to_call(
                state_abbr=function_args.get("state_abbr"),
                specific_date=function_args.get("specific_date"),
            )
            print(function_response)
            print(tool_call.id)
            tool_outputs.append(
                { "tool_call_id": tool_call.id,
                 "output": str(function_response)
                }
            )

        run = client.beta.threads.runs.submit_tool_outputs(
          thread_id=thread.id,
          run_id=run.id,
          tool_outputs = tool_outputs
        )


messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

print(messages)

Elapsed time: 0 minutes 5 seconds
Status: requires_action
{'date': '2021-03-05', 'hospitalizedIncrease': 3}
call_6FnFh3BGcIkglAP9p64AtAJR
Elapsed time: 0 minutes 11 seconds
Status: completed
SyncCursorPage[ThreadMessage](data=[ThreadMessage(id='msg_r6iES2TrwtZVqvB5KZQs29JQ', assistant_id='asst_9G5fSzNkdqRnsHIRXw3gfr4g', content=[MessageContentText(text=Text(annotations=[], value='On March 5, 2021, there were 3 new hospitalizations reported in Alaska due to COVID-19.'), type='text')], created_at=1718639665, file_ids=[], metadata={}, object='thread.message', role='assistant', run_id='run_ow6Y1YbuZ4fP0cbTeJ5AdaC5', thread_id='thread_wsxhkqX5xbIs0l4Aigl4OpsF'), ThreadMessage(id='msg_j5e3kwMb3vnn5drHD2yjDVgF', assistant_id=None, content=[MessageContentText(text=Text(annotations=[], value='how many hospitalized people we had in Alaska\n               the 2021-03-05?'), type='text')], created_at=1718639643, file_ids=[], metadata={}, object='thread.message', role='user', run_id=None, thread_id

In [8]:
print(messages.model_dump_json(indent=2))

{
  "data": [
    {
      "id": "msg_r6iES2TrwtZVqvB5KZQs29JQ",
      "assistant_id": "asst_9G5fSzNkdqRnsHIRXw3gfr4g",
      "content": [
        {
          "text": {
            "annotations": [],
            "value": "On March 5, 2021, there were 3 new hospitalizations reported in Alaska due to COVID-19."
          },
          "type": "text"
        }
      ],
      "created_at": 1718639665,
      "file_ids": [],
      "metadata": {},
      "object": "thread.message",
      "role": "assistant",
      "run_id": "run_ow6Y1YbuZ4fP0cbTeJ5AdaC5",
      "thread_id": "thread_wsxhkqX5xbIs0l4Aigl4OpsF"
    },
    {
      "id": "msg_j5e3kwMb3vnn5drHD2yjDVgF",
      "assistant_id": null,
      "content": [
        {
          "text": {
            "annotations": [],
            "value": "how many hospitalized people we had in Alaska\n               the 2021-03-05?"
          },
          "type": "text"
        }
      ],
      "created_at": 1718639643,
      "file_ids": [],
      "metadata": 

## Add the code interpreter

In [10]:
file = client.files.create(
  file=open("./data/all-states-history.csv", "rb"),
  purpose='assistants'
)
assistant = client.beta.assistants.create(
  instructions="""You are an assitant answering questions about
                  a Covid dataset.""",
  model="gpt-4-1106", 
  tools=[{"type": "code_interpreter"}],
  file_ids=[file.id])
thread = client.beta.threads.create()
print(thread)
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="""how many hospitalized people we had in Alaska
               the 2021-03-05?"""
)
print(message)
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
)

Thread(id='thread_fuxOC0lYpnBmPx7kWAFxk7qx', created_at=1718639691, metadata={}, object='thread')
ThreadMessage(id='msg_XuQA5tN8MNRiAfk1EP61kSjV', assistant_id=None, content=[MessageContentText(text=Text(annotations=[], value='how many hospitalized people we had in Alaska\n               the 2021-03-05?'), type='text')], created_at=1718639691, file_ids=[], metadata={}, object='thread.message', role='user', run_id=None, thread_id='thread_fuxOC0lYpnBmPx7kWAFxk7qx')


In [11]:
status = run.status
start_time = time.time()
while status not in ["completed", "cancelled", "expired", "failed"]:
    time.sleep(5)
    run = client.beta.threads.runs.retrieve(
        thread_id=thread.id,
        run_id=run.id
    )
    print("Elapsed time: {} minutes {} seconds".format(
        int((time.time() - start_time) // 60),
        int((time.time() - start_time) % 60))
         )
    status = run.status
    print(f'Status: {status}')
    clear_output(wait=True)


messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

print(messages.model_dump_json(indent=2))

{
  "data": [
    {
      "id": "msg_LN2cxoOMP4tR84wPndEs1QW8",
      "assistant_id": "asst_z26RAcdYn4BqfVy7K6qwaoDB",
      "content": [
        {
          "text": {
            "annotations": [],
            "value": "On March 5, 2021, there were 33 people hospitalized in Alaska due to COVID-19."
          },
          "type": "text"
        }
      ],
      "created_at": 1718639718,
      "file_ids": [],
      "metadata": {},
      "object": "thread.message",
      "role": "assistant",
      "run_id": "run_q5eZxwA3YCV82c131DhxgMuq",
      "thread_id": "thread_fuxOC0lYpnBmPx7kWAFxk7qx"
    },
    {
      "id": "msg_RM1Hhvqvj4fxHEX6tplBDQ0f",
      "assistant_id": "asst_z26RAcdYn4BqfVy7K6qwaoDB",
      "content": [
        {
          "text": {
            "annotations": [],
            "value": "The dataset has been successfully loaded. There are several columns related to hospitalization, such as `hospitalized`, `hospitalizedCumulative`, and `hospitalizedCurrently`. To answer your 